# Create Azure SQL Managed Instance

## Description

This PowerShell script example creates a managed instance in a dedicated subnet within a new virtual network. It also configures a route table and a network security group for the virtual network. Once the script has been successfully run, the managed instance can be accessed from within the virtual network or from an on-premises environment. See [Configure Azure VM to connect to Azure SQL Database Managed Instance](https://docs.microsoft.com/en-us/azure/azure-sql/managed-instance/connect-vm-instance-configure) and [Configure a point-to-site connection to Azure SQL Managed Instance from on-premises](https://docs.microsoft.com/en-us/azure/azure-sql/managed-instance/point-to-site-p2s-configure).

<img width="50%" src=https://docs.microsoft.com/en-us/azure/sql-database/media/sql-database-managed-instance/key-features.png />

Steps of this procedure include:
1. Connect to Azure subscription
1. Provision resource group for SQL Managed Instance
2. Configure Network Settings
3. Create credentials to be used for Managed Instance
4. Create Managed Instance

In [ ]:
$NSnetworkModels = "Microsoft.Azure.Commands.Network.Models"
$NScollections = "System.Collections.Generic"

# Set the resource group name and Location for your managed instance
$ResourceGroupName = "myResourceGroup-$(Get-Random)"
$Location = "eastus2"

# Set the networking values for your managed instance
$VNetName = "myVnet-$(Get-Random)"
$VNetAddressPrefix = "10.0.0.0/16"
$DefaultSubnetName = "myDefaultSubnet-$(Get-Random)"
$DefaultSubnetAddressPrefix = "10.0.0.0/24"
$MiSubnetName = "myMISubnet-$(Get-Random)"
$MiSubnetAddressPrefix = "10.0.0.0/24"

#Set the managed instance name for the new managed instance
$InstanceName = "myMIName-$(Get-Random)"
# Set the admin login and password for your managed instance
$MiAdminSqlLogin = "SqlAdmin"
$MiAdminSqlPassword = "ChangeYourAdminPassword1"

# Set the managed instance service tier, compute level, and License mode
$Edition = "General Purpose"
$VCores = 8
$MaxStorage = 256
$ComputeGeneration = "Gen5"
$License = "LicenseIncluded" #"BasePrice" or LicenseIncluded if you have don't have SQL Server licence that can be used for AHB discount


## Connect to Azure Account

In [ ]:
Connect-AzAccount

## Get Subscription
Below command will open a Dialouge Box with list of subscriptions.
Selecting one of those will set that subscription for rest of the commands.

In [ ]:
$subscription = Get-AzSubscription | Out-GridView -PassThru
Set-AzContext -SubscriptionName $subscription

## Create the resource group
Create a resource group with the _New-AzResourceGroup_ command. An Azure resource group is a logical container into which Azure resources are deployed and managed. A resource group must be created first:

In [ ]:
# Create Azure resource group, if necessary
$resourceGroup = Get-AzResourceGroup -Name $ResourceGroupName

if (!$resourceGroup)
{
    # Need to create a new resource group
    Write-Output "Resource Group $RG1 does not exist. Creating..."
    $resourceGroup = New-AzResourceGroup -Name $RG1 -Location $Location -Tag @{Owner="SQLDB-Samples"}
}

## Configure virtual network, subnets, network security group, and routing table

In [ ]:
$networkSecurityGroupMiManagementService = New-AzNetworkSecurityGroup `
                      -Name 'myNetworkSecurityGroupMiManagementService' `
                      -ResourceGroupName $ResourceGroupName `
                      -Location $Location

$routeTableMiManagementService = New-AzRouteTable `
                      -Name 'myRouteTableMiManagementService' `
                      -ResourceGroupName $ResourceGroupName `
                      -Location $Location

$virtualNetwork = New-AzVirtualNetwork `
                      -ResourceGroupName $ResourceGroupName `
                      -Location $Location `
                      -Name $VNetName `
                      -AddressPrefix $VNetAddressPrefix

                  Add-AzVirtualNetworkSubnetConfig `
                      -Name $MiSubnetName `
                      -VirtualNetwork $virtualNetwork `
                      -AddressPrefix $MiSubnetAddressPrefix `
                      -NetworkSecurityGroup $networkSecurityGroupMiManagementService `
                      -RouteTable $routeTableMiManagementService |
                  Set-AzVirtualNetwork

$virtualNetwork = Get-AzVirtualNetwork -Name $VNetName -ResourceGroupName $ResourceGroupName

$subnet= $virtualNetwork.Subnets[0]

## Create a delegation

In [ ]:
$subnet.Delegations = New-Object "$NScollections.List``1[$NSnetworkModels.PSDelegation]"
$delegationName = "dgManagedInstance" + (Get-Random -Maximum 1000)
$delegation = New-AzDelegation -Name $delegationName -ServiceName "Microsoft.Sql/managedInstances"
$subnet.Delegations.Add($delegation)

Set-AzVirtualNetwork -VirtualNetwork $virtualNetwork

$miSubnetConfigId = $subnet.Id



$allowParameters = @{
    Access = 'Allow'
    Protocol = 'Tcp'
    Direction= 'Inbound'
    SourcePortRange = '*'
    SourceAddressPrefix = 'VirtualNetwork'
    DestinationAddressPrefix = '*'
}
$denyInParameters = @{
    Access = 'Deny'
    Protocol = '*'
    Direction = 'Inbound'
    SourcePortRange = '*'
    SourceAddressPrefix = '*'
    DestinationPortRange = '*'
    DestinationAddressPrefix = '*'
}
$denyOutParameters = @{
    Access = 'Deny'
    Protocol = '*'
    Direction = 'Outbound'
    SourcePortRange = '*'
    SourceAddressPrefix = '*'
    DestinationPortRange = '*'
    DestinationAddressPrefix = '*'
}

Get-AzNetworkSecurityGroup `
        -ResourceGroupName $ResourceGroupName `
        -Name "myNetworkSecurityGroupMiManagementService" |
    Add-AzNetworkSecurityRuleConfig `
        @allowParameters `
        -Priority 1000 `
        -Name "allow_tds_inbound" `
        -DestinationPortRange 1433 |
    Add-AzNetworkSecurityRuleConfig `
        @allowParameters `
        -Priority 1100 `
        -Name "allow_redirect_inbound" `
        -DestinationPortRange 11000-11999 |
    Add-AzNetworkSecurityRuleConfig `
        @denyInParameters `
        -Priority 4096 `
        -Name "deny_all_inbound" |
    Add-AzNetworkSecurityRuleConfig `
        @denyOutParameters `
        -Priority 4096 `
        -Name "deny_all_outbound" |
    Set-AzNetworkSecurityGroup

## Create credentials

In [ ]:
$secpassword = ConvertTo-SecureString $MiAdminSqlPassword -AsPlainText -Force
$credential = New-Object System.Management.Automation.PSCredential ($MiAdminSqlLogin, $secpassword)


## Create managed instance

### This script will take a minimum of 3 hours to create a new managed instance in a new virtual network. 
### A second managed instance is created much faster.

In [ ]:
New-AzSqlInstance -Name $InstanceName `
                      -ResourceGroupName $ResourceGroupName -Location $Location -SubnetId $miSubnetConfigId `
                      -AdministratorCredential $credential `
                      -StorageSizeInGB $MaxStorage -VCore $VCores -Edition $Edition `
                      -ComputeGeneration $ComputeGeneration -LicenseType $License
